In [1]:
import sys
sys.path.append('/opt/ml/backup')
from load_data import load_data
from googletrans import Translator

from tqdm import tqdm
from typing import Optional

In [2]:
data = load_data("/opt/ml/input/data/train/train_renew.tsv")
translator = Translator() # 번역을 위한 객체

In [7]:
def augment_sentence(record: dict, lang: str = 'en') -> Optional[dict]:
    entity_code01 = "ZQWXEC" # entity가 사라지는 것을 방지
    entity_code02 = "QZWXEC"
    
    sentence = record['sentence']
    entity01 = record['entity_01']
    entity02 = record['entity_02']

    sentence = sentence.replace(entity01, entity_code01).replace(entity02, entity_code02)

    try:
        result_trans = translator.translate(sentence, src='ko', dest=lang)
        result_trans_back = translator.translate(result_trans.text, src=lang, dest='ko')

        aug_sentence = result_trans_back.text.replace(entity_code01, entity01).replace(entity_code02, entity02)
    except:
        return None
    
    if entity01 in aug_sentence and entity02 in aug_sentence:    
        return {
            "sentence": aug_sentence,
            "entity_01": entity01,
            "entity_02": entity02,
            "label": record['label']
        }
    else:
        return None

In [3]:
target_langs = ['en']

aug_sentences = []
for i in tqdm(range(data.shape[0]), desc="Augmenting ..."):
    if data.iloc[i]['label'] != 0:
        aug_sentences += [augment_sentence(data.iloc[i].to_dict(), lang=t_lang) for t_lang in target_langs]
        
aug_sentences = [record for record in aug_sentences if record is not None]

In [19]:
aug_data = data.append(aug_sentences)

aug_data.to_csv("/opt/ml/input/data/train/aug_train.tsv", index=False,header = None, sep = "\t")

In [4]:
print(len(aug_data))